In [59]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,f1_score,roc_curve
import joblib
import os

# Logistic Regression
def logisticRegression(trainX, trainY, testX, testY, decision_boundary=0.5):
    '''
        @Input
            trainX : type : array
            trainY : type : array
            testX : type : array
            testY : type : array

            decision_boundary : type : float (0~1사이의 값)
        @Output
            AucScore : type : float 64
            f1 : type : float 64
            precision : type : float 64
            recall : type : float 64
            confusionmatrix : array
    '''
    model = LogisticRegression(random_state=0).fit(trainX, trainY)

    # Logistic Regression 모델 Pickle 파일로 저장

    saveModel = joblib.dump(model, 'Logistic.pkl')

    anomalyScore = model.predict_proba(testX)
    #[ [1, 0]  ]
    pred = np.array([], dtype=np.int32)

    for i in range(len(anomalyScore)):
        if (anomalyScore[i][1] > decision_boundary) == True:
            pred = np.append(pred, 1)
        else:
            pred = np.append(pred, 0)

    cofMat = confusion_matrix(testY, pred, labels=[1, 0])  # 1: 불량, 0: 정상
    tpr = cofMat[1, 1] / (cofMat[1, 1] + cofMat[1, 0])
    fpr = cofMat[0, 1] / (cofMat[0, 0] + cofMat[0, 1])
    
    fpr_array = np.array([], dtype=np.int32)
    tpr_array = np.array([], dtype=np.int32)

    # Decision Boundary 별 FPR, TPR 계산 (AUC Score 시각화 용도 )
    for i in range(1, 100):
        predd = np.array([], dtype=np.int32)
        for j in range(len(anomalyScore)):
            decision_boundary = i / 100
            if (anomalyScore[j][1] > decision_boundary) == True:
                predd = np.append(predd, 1)
            else:
                predd = np.append(predd, 0)
        cofMatt = confusion_matrix(testY, predd, labels=[1, 0])  # 1: 불량, 0: 정상
        
        ##confusion matrix의 각 요소
        auc_tpr = cofMatt[1, 1] / (cofMatt[1, 1] + cofMatt[1, 0])
        auc_fpr = cofMatt[0, 1] / (cofMatt[0, 0] + cofMatt[0, 1])
        fpr_array = np.append(fpr_array, auc_fpr)
        tpr_array = np.append(tpr_array, auc_tpr)

    # calculate AUC of model
    auc = roc_auc_score(testY, pred)
    return ({"AnomalyScore": np.round(anomalyScore[:,1],3),"Prediction":pred ,"Aucscore": str(auc), "Fpr": fpr, "Tpr": tpr, "Fprarray":fpr_array, "Tprarray":tpr_array})
              # "accuracy": str(accuracy), "precision": str(precision), "recall": str(recall), "f1": str(f1), "tp": str(tp), "fp": str(fp), "fn": str(fn), "Predarray" :predarray,
              # "tn": str(tn)})  # type : int32

def classification_model_load_logistic(model_wd, testX, decision_boundary=0.5):
    model_wd = 'Logistic.pkl'
    model = joblib.load(model_wd)
    anomalyScore = model.predict_proba(testX)
    pred = np.array([], dtype=np.int32)
    
    for i in range(len(anomalyScore)):
        if (anomalyScore[i][1] > decision_boundary) == True:
            pred = np.append(pred, 1)
        else:
            pred = np.append(pred, 0)

    return {"AnomalyScore": np.round(anomalyScore,3), "Prediction":pred}


In [25]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris(as_frame=True)

In [36]:
df = pd.read_csv('/workspace/xnsolution/data/1_Iris_virginica.csv', encoding='euc-kr')

df['label'][np.where(df['label'] == 'outlier')[0]] = 1
df['label'][np.where(df['label'] == 'target')[0]] = 0

X = df.iloc[:,0:4]
y = df.iloc[:,4]
y= y.astype('category')

trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.3)

# model = classification_model_load_logistic(model, testX, decision_boundary=0.5)
# load_model = classification_model_load_logistic('Logistic.pkl', testX, decision_boundary=0.5)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as pydatetime
import os

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix,f1_score,roc_curve
from sklearn import tree
import joblib



    def classification_decisiontree(trainX, testX, trainY, testY, criterion='gini', max_depth=None, min_samples_leaf=1):
        """

        Parameters
        ----------
        trainX : array
            Train X.
        testX : array
            Test X.
        trainY : array
            Train Y.
        testY : array
            Test Y.
        criterion : {'gini', 'entropy', 'log_loss'}, default = "gini"
            의사 결정 나무의 분기 기준.
        max_depth : int, optional, default = None
            의사 결정 나무의 최대 깊이. 의사 결정 나무가 최대 깊이에 도달하면 더 이상 분기하지 않음.
        min_samples_leaf : int or float, optional, default = 2
            노드 별 최소 샘플 갯수, 터미널 노드안에 샘플 수가 min_samples_split과 같아지면 더 이상 분기하지 않음.

        Returns
        -------
        model :
            분류 의사결정나무 모델.
        confusionMatrix : array
            분류 성능 평가를 위한 Confusion Matrix
        fpr : array
            False Positive Rate
        tpr : array
            True Positive Rate
        auc_score : float
            fpr, tpr의 AUC(Area Under Cruve) scroe (0~1). 1에 가까울 수록 분류 성능이 좋음을 의미함
        accuracy : float
            accuracy.
        precision : float
            precision
        recall : float
            recall
        f1 : float
            f1 score
        tp : int
            True positive
        fp : int
            False positive
        fn : int
            False negative
        tn : int
            True negative
        """

        trainnX = trainX

        if isinstance(trainX, pd.DataFrame):
            trainX = trainX.values
        if isinstance(testX, pd.DataFrame):
            testX = testX.values
        if isinstance(trainY, pd.DataFrame):
            trainY = trainY.values
        if isinstance(testY, pd.DataFrame):
            testY = testY.values

        # Decision Tree Model
        model = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=min_samples_leaf)
        model.fit(trainX, trainY)

        # Decision Tree 모델 Pickle 파일로 저장
        modelfilename = Cmm.GetSaveModelName("CFDecisionTree")
        modelpath = Cmm.GetSaveModelPath() + modelfilename
        saveModel = joblib.dump(model, modelpath)
        modelstr64 = Cmm.FileToBase64(modelpath)

        if os.path.exists(modelpath):
            os.remove(modelpath)

        # 예측
        anomalyScore = model.predict_proba(testX)  # 예측 분류 값
        pred = model.predict(testX)

        predarray = Cmm.ListToOneValueArraystring(pred)

        # 분류 평가 지표
        cofMat = confusion_matrix(testY, pred, labels=[1, 0])  # 1: 불량, 0: 정상
        # confusion matrix의 각 요소
        tp = cofMat[1, 1]
        tn = cofMat[0, 0]
        fn = cofMat[1, 0]
        fp = cofMat[0, 1]

        accuracy = accuracy_score(testY, pred)
        precision = precision_score(testY, pred)
        recall = recall_score(testY, pred)
        f1 = f1_score(testY, pred)

        # calculate AUC of model
        auc = roc_auc_score(testY, pred)


        fpr_array = np.array([], dtype=np.int32)
        tpr_array = np.array([], dtype=np.int32)
        for i in range(1, 100):
            predd = np.array([], dtype=np.int32)
            for j in range(len(anomalyScore)):
                decision_boundary = i / 100
                if (anomalyScore[j][1] > decision_boundary) == True:
                    predd = np.append(predd, 1)
                else:
                    predd = np.append(predd, 0)

            cofMatt = confusion_matrix(testY, predd, labels=[1, 0])  # 1: 불량, 0: 정상
            ##confusion matrix의 각 요소
            tpp = cofMatt[1, 1]
            tnn = cofMatt[0, 0]
            fnn = cofMatt[1, 0]
            fpp = cofMatt[0, 1]
            tpr = tpp / (tpp + fnn)
            fpr = fpp / (tnn + fpp)

            fpr_array = np.append(fpr_array, fpr)
            tpr_array = np.append(tpr_array, tpr)

        fpr, tpr, thresholds = roc_curve(testY, pred)

        lists = fpr.tolist()
        fprJson = Cmm.NumlistToArraystring(lists)
        lists = tpr.tolist()
        tprJson = Cmm.NumlistToArraystring(lists)

        lists = cofMat.tolist()
        cofMatJson = Cmm.NumlistToArraystring(lists)
        lists = fpr_array.tolist()
        fpr_arrayJson = Cmm.NumlistToArraystring(lists)
        lists = tpr_array.tolist()
        tpr_arrayJson = Cmm.NumlistToArraystring(lists)

        anomalyScore = np.round(anomalyScore[:, 1], 3)
        lists = anomalyScore.tolist()
        anomalyScoreJson = Cmm.NumlistToArraystring(lists)

        plt.rc('font', family='Malgun Gothic')
        if max_depth == None:
            plt.figure(figsize=(10, 8))
        else:
            plt.figure(figsize=(4 * max_depth, 2 * max_depth))
        tree.plot_tree(model,
                       feature_names=trainnX.columns,
                       impurity=True, filled=True,
                       rounded=True, max_depth=max_depth)

        fname = ('C:/Temp/' + str(pydatetime.datetime.now().timestamp()) + ".png")
        plt.savefig(fname, dpi=100)
        str64 = Cmm.FileToBase64(fname)

        if os.path.exists(fname):
            os.remove(fname)


        return {"AnomalyScore":anomalyScoreJson, "Aucscore": str(auc), "Fpr": fprJson, "Tpr": tprJson,"Fprarray": fpr_arrayJson, "Tprarray": tpr_arrayJson, "Predarray" :predarray}, {"Value" :str(str64),"Modelfile": str(modelfilename), "Modelcontents": str(modelstr64)}
                 # "model": str(model),  "accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "tp": tp, "fp": fp, "fn": fn,
                 # "tn": tn})  # type : int32

def classification_model_load_DT(model, testX):
    anomalyScore = model.predict_proba(testX)  # 예측 분류 값
    pred = model.predict(testX)

    predarray = Cmm.ListToOneValueArraystring(pred)

    # # 분류 평가 지표
    # cofMat = confusion_matrix(testY, pred, labels=[1, 0])  # 1: 불량, 0: 정상
    # # confusion matrix의 각 요소
    # tp = cofMat[1, 1]
    # tn = cofMat[0, 0]
    # fn = cofMat[1, 0]
    # fp = cofMat[0, 1]
    #
    # accuracy = accuracy_score(testY, pred)
    # precision = precision_score(testY, pred)
    # recall = recall_score(testY, pred)
    # f1 = f1_score(testY, pred)
    #
    # # calculate AUC of model
    # auc = roc_auc_score(testY, pred)
    #
    # fpr_array = np.array([], dtype=np.int32)
    # tpr_array = np.array([], dtype=np.int32)
    # for i in range(1, 100):
    #     predd = np.array([], dtype=np.int32)
    #     for j in range(len(pred_prob)):
    #         decision_boundary = i / 100
    #         if (pred_prob[j][1] > decision_boundary) == True:
    #             predd = np.append(predd, 1)
    #         else:
    #             predd = np.append(predd, 0)
    #
    #     cofMatt = confusion_matrix(testY, predd, labels=[1, 0])  # 1: 불량, 0: 정상
    #     ##confusion matrix의 각 요소
    #     tpp = cofMatt[1, 1]
    #     tnn = cofMatt[0, 0]
    #     fnn = cofMatt[1, 0]
    #     fpp = cofMatt[0, 1]
    #     tpr = tpp / (tpp + fnn)
    #     fpr = fpp / (tnn + fpp)
    #
    #     fpr_array = np.append(fpr_array, fpr)
    #     tpr_array = np.append(tpr_array, tpr)
    #
    # fpr, tpr, thresholds = roc_curve(testY, pred)
    #
    # lists = fpr.tolist()
    # fprJson = Cmm.NumlistToArraystring(lists)
    # lists = tpr.tolist()
    # tprJson = Cmm.NumlistToArraystring(lists)
    #
    # lists = cofMat.tolist()
    # cofMatJson = Cmm.NumlistToArraystring(lists)
    # lists = fpr_array.tolist()
    # fpr_arrayJson = Cmm.NumlistToArraystring(lists)
    # lists = tpr_array.tolist()
    # tpr_arrayJson = Cmm.NumlistToArraystring(lists)
    #
    # plt.rc('font', family='Malgun Gothic')
    # if max_depth == None:
    #     plt.figure(figsize=(10, 8))
    # else:
    #     plt.figure(figsize=(4 * max_depth, 2 * max_depth))
    # tree.plot_tree(model,
    #                feature_names=trainnX.columns,
    #                impurity=True, filled=True,
    #                rounded=True, max_depth=max_depth)
    #
    # fname = ('C:/Temp/' + str(pydatetime.datetime.now().timestamp()) + ".png")
    # plt.savefig(fname, dpi=100)
    # str64 = Cmm.FileToBase64(fname)
    #
    # if os.path.exists(fname):
    #     os.remove(fname)

    return {"Predarray": predarray}
    # return {"Aucscore": str(auc), "Fpr": fprJson, "Tpr": tprJson, "Fprarray": fpr_arrayJson, "Tprarray": tpr_arrayJson,
    #         "Predarray": predarray}, {"Value": str(str64), "Modelfile": str(modelfilename), "Modelcontents": str(str64)}
    # "model": str(model),  "accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "tp": tp, "fp": fp, "fn": fn,
    # "tn": tn})  # type : int32

# a = Classification.decisiontreeexcute('SET2022000007-CS','SET2022000007-CT', '계량완료위치,사출최대속도','판정코드','gini',3,2)
# print(a)

# ##예제
# # import pandas as pd
# ab_tr = pd.read_csv('C:\\Classfication\\ab_tr.csv')
# ab_te = pd.read_csv('C:\\Classfication\\ab_te.csv')
# # #
# # #ab_tr = ab_tr.drop(['Unnamed: 0'],axis=1)
# # #ab_te = ab_te.drop(['Unnamed: 0'],axis=1)
# # #
# ab_tr_x = ab_tr.drop(['target'],axis=1)
# ab_te_x = ab_te.drop(['target'],axis=1)
# print(ab_tr_x)
# print(ab_te_x)
# ab_tr_y = ab_tr['target']
# ab_te_y = ab_te['target']
# #trainX = ab_tr_x
# #trainY = ab_tr_y
#
# #testX = ab_te_x
# #testY = ab_te_y
#
# # # # print(ab_tr_y)
# # # # print(ab_te_y)
# #
# ab_dt = classification_decisiontree(ab_tr_x, ab_te_x, ab_tr_y, ab_te_y,max_depth=3)
# print(ab_dt)


#DT_model = joblib.load()
#print(classification_model_load_DT(DT_model,ab_te_x,ab_te_y))

In [ ]:
import pandas as pd
import numpy as np
from XNCmmLib import XNCmmUnit as Cmm
import XNDatabaseLib as DbLib
import os

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix,f1_score,roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import joblib

class Classification:

    def knnexcute(trdataID, tsdataID, cols, fcols,n_neighbors):
        # TrainingID의 cols열을 dataframe 변수
        txt = DbLib.DatabaseUnit.GetDataSet(trdataID)
        df = pd.read_csv(Cmm.StrToStringIO(txt), sep=',', encoding='utf-8-sig')
        trdf = Cmm.DfToCustomDf(df, Cmm.StrToList(cols, ','))
        # print(trdf)
        # 예측하는 데이터열을 dataframe 변수
        trfdf = Cmm.DfToCustomDf(df, Cmm.StrToList(fcols, ','))
        # print(trfdf)
        txt = DbLib.DatabaseUnit.GetDataSet(tsdataID)
        df = pd.read_csv(Cmm.StrToStringIO(txt), sep=',', encoding='utf-8-sig')
        tsdf = Cmm.DfToCustomDf(df, Cmm.StrToList(cols, ','))
        # print(tsdf)
        tsfdf = Cmm.DfToCustomDf(df, Cmm.StrToList(fcols, ','))
        # print(tsfdf)
        # cbm 호출
        a = Classification.knnClassifier(trdf, trfdf ,tsdf, tsfdf, n_neighbors)
        jstr = Cmm.ClassToJson(a)

        return jstr

    # KNN (Classification)
    def knnClassifier(x_train, y_train, x_test, y_test, number_neighbors):
        '''
        @Input
            x_train : type : array
            y_train : type : array
            x_test : type : array
            n_neighbors : type : int
        @Output
            auc_score : type : float64
            fpr : type : array
            fpr : type : array
        '''

        if isinstance(x_train, pd.DataFrame):
            x_train = x_train.values
        if isinstance(x_test, pd.DataFrame):
            x_test = x_test.values
        if isinstance(y_train, pd.DataFrame):
            y_train = y_train.values
        if isinstance(y_test, pd.DataFrame):
            y_test = y_test.values

        model = KNeighborsClassifier(n_neighbors=number_neighbors)
        model.fit(x_train, y_train)
        anomalyScore = model.predict_proba(x_test)

        # # KNN 모델 Pickle 파일로 저장
        modelfilename = Cmm.GetSaveModelName("CFKnn")
        modelpath = Cmm.GetSaveModelPath() + modelfilename
        saveModel = joblib.dump(model, modelpath)
        str64 = Cmm.FileToBase64(modelpath)

        if os.path.exists(modelpath):
            os.remove(modelpath)


        y_pred = model.predict(x_test)  # 불량 : 1, 정상 : 0

        predarray = Cmm.ListToOneValueArraystring(y_pred)

        # calculate AUC of model
        auc = roc_auc_score(y_test, y_pred)


        fpr_array = np.array([], dtype=np.int32)
        tpr_array = np.array([], dtype=np.int32)

        for i in range(1, 100):
            predd = np.array([], dtype=np.int32)
            for j in range(len(anomalyScore)):
                decision_boundary = i / 100
                if (anomalyScore[j][1] > decision_boundary) == True:
                    predd = np.append(predd, 1)
                else:
                    predd = np.append(predd, 0)

            cofMatt = confusion_matrix(y_test, predd, labels=[1, 0])  # 1: 불량, 0: 정상
            ##confusion matrix의 각 요소
            tpp = cofMatt[1, 1]
            tnn = cofMatt[0, 0]
            fnn = cofMatt[1, 0]
            fpp = cofMatt[0, 1]
            tpr = tpp / (tpp + fnn)
            fpr = fpp / (tnn + fpp)

            fpr_array = np.append(fpr_array, fpr)
            tpr_array = np.append(tpr_array, tpr)

        fpr, tpr, thresholds = roc_curve(y_test, y_pred)

        lists = fpr.tolist()
        fprJson = Cmm.NumlistToArraystring(lists)
        lists = tpr.tolist()
        tprJson = Cmm.NumlistToArraystring(lists)

        anomalyScore = np.round(anomalyScore[:, 1], 3)
        lists = anomalyScore.tolist()
        anomalyScoreJson = Cmm.NumlistToArraystring(lists)

        fpr_array = Cmm.ListToOneValueArraystring(fpr_array)
        tpr_array = Cmm.ListToOneValueArraystring(tpr_array)


        return ({"AnomalyScore" : anomalyScoreJson, "Aucscore": str(auc), "Fpr": fprJson, "Tpr": tprJson, "Fprarray":fpr_array, "Tprarray":tpr_array, "Predarray" : predarray}), {"Modelfile":str(modelfilename),"Modelcontents": str(str64)}

def classification_model_load_KN(model, x_test):
    anomalyScore = model.predict_proba(x_test)
    y_pred = model.predict(x_test)  # 불량 : 1, 정상 : 0

    predarray = Cmm.ListToOneValueArraystring(y_pred)

    # # calculate AUC of model
    # auc = roc_auc_score(y_test, y_pred)
    #
    # fpr_array = np.array([], dtype=np.int32)
    # tpr_array = np.array([], dtype=np.int32)
    #
    # for i in range(1, 100):
    #     predd = np.array([], dtype=np.int32)
    #     for j in range(len(pred_prob)):
    #         decision_boundary = i / 100
    #         if (pred_prob[j][1] > decision_boundary) == True:
    #             predd = np.append(predd, 1)
    #         else:
    #             predd = np.append(predd, 0)
    #
    #     cofMatt = confusion_matrix(y_test, predd, labels=[1, 0])  # 1: 불량, 0: 정상
    #     ##confusion matrix의 각 요소
    #     tpp = cofMatt[1, 1]
    #     tnn = cofMatt[0, 0]
    #     fnn = cofMatt[1, 0]
    #     fpp = cofMatt[0, 1]
    #     tpr = tpp / (tpp + fnn)
    #     fpr = fpp / (tnn + fpp)
    #
    #     fpr_array = np.append(fpr_array, fpr)
    #     tpr_array = np.append(tpr_array, tpr)
    #
    # fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    #
    # lists = fpr.tolist()
    # fprJson = Cmm.NumlistToArraystring(lists)
    # lists = tpr.tolist()
    # tprJson = Cmm.NumlistToArraystring(lists)
    #
    # fpr_array = Cmm.ListToOneValueArraystring(fpr_array)
    # tpr_array = Cmm.ListToOneValueArraystring(tpr_array)

    # str64 = Cmm.FileToBase64(modelpath)

    return {"Predarray": predarray}
    # return ({"Aucscore": str(auc), "Fpr": fprJson, "Tpr": tprJson, "Fprarray": fpr_array, "Tprarray": tpr_array,
    #          "Predarray": predarray}), {"Modelfile": str(modelfilename), "Modelcontents": str(str64)}


# a = Classification.knnexcute('SET2022000017-CS','SET2022000017-CT', '계량완료위치,사출최대속도,사출최대압력','판정코드',3)
# print(a)

# ##예제

# train = pd.read_csv("C:\\Classfication\\ab_tr.csv")
# test = pd.read_csv("C:\\Classfication\\ab_te.csv")
#
# x_train = train.values[:,0:4]
# y_train = train.values[:,4]
#
# x_test = test.values[:,0:4]
# y_test = test.values[:,4]
# # #
# knn = Classification.knnClassifier(x_train, y_train, x_test, y_test, 3)
# print(knn)

#knn_model = joblib.load()
#print(classification_model_load_KN(knn_model,x_test,y_test))

# pickle file load
#knnpickle = joblib.load("C:\\ModelStorage\\220718102310792375_CFKnn.pkl")

#print(knnpickle.)


#knn_model = joblib.load()
#print(classification_model_load_KN(knn_model,x_test,y_test))


In [ ]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd
from XNCmmLib import XNCmmUnit as Cmm
import XNDatabaseLib as DbLib
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix,f1_score,roc_curve
import joblib
import os

# NuralNet (Classification)
class ClassificationNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__()  # nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1), node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2, hiddenlayer + 2):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer + 2)] = nn.Linear(node, Ouputnode)
        self.layer1 = nn.Sequential(ordered_dict)
        # print(self.layer1)

    def predict(self, X_test):
        pred_x = self.layer1(X_test)
        pred_x = pred_x.detach().numpy()
        pred_x = pred_x.tolist()
        pred_x = np.array(pred_x)

        return {"Predarray" :pred_x}

    def fit(self, epochs, loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                return
            # print("{}epoch // loss ={}".format(epoch, loss))

class Classification:
    def cnetexcute(trdataID, tsdataID, cols, fcols, hiddenlayer, node, Ouputnode, epochs):
        # TrainingID의 cols열을 dataframe 변수
        txt = DbLib.DatabaseUnit.GetDataSet(trdataID)
        df = pd.read_csv(Cmm.StrToStringIO(txt), sep=',', encoding='utf-8-sig')
        trdf = Cmm.DfToCustomDf(df, Cmm.StrToList(cols, ','))
        # print(trdf)
        # 예측하는 데이터열을 dataframe 변수
        trfdf = Cmm.DfToCustomDf(df, Cmm.StrToList(fcols, ','))
        # print(trfdf)
        txt = DbLib.DatabaseUnit.GetDataSet(tsdataID)
        df = pd.read_csv(Cmm.StrToStringIO(txt), sep=',', encoding='utf-8-sig')
        tsdf = Cmm.DfToCustomDf(df, Cmm.StrToList(cols, ','))
        # print(tsdf)
        tsfdf = Cmm.DfToCustomDf(df, Cmm.StrToList(fcols, ','))
        # print(tsfdf)

        # cnet 호출
        a = Classification.CNet(trdf, trfdf, tsdf, tsfdf, hiddenlayer, node, Ouputnode, epochs)

        jstr = Cmm.ClassToJson(a)

        return jstr


    def CNet(X_train, y_train, X_test, y_test, hiddenlayer, node, Ouputnode, epochs):
        '''

            @Input
                X_train : type : array
                y_train : type : array
                X_test : type : array
                hiddenlayer : type : int
                node : type : int
                Ouputnode : type : int
                epochs : type : int

            @Output
                pred_y_test : type : array

        '''

        # 데이터를 파이토치 텐서로 변경
        X = torch.Tensor(X_train.values)
        Y = torch.Tensor(y_train.values)
        X_test = torch.Tensor(X_test.values)

        # 학습을 위한 데이터 전처리
        dataset = TensorDataset(X, Y)
        loader = DataLoader(dataset, batch_size=1, shuffle=True)

        # 신경망 생성하기
        model = ClassificationNet(X, hiddenlayer, node, Ouputnode)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.001)
        model.fit(epochs, loader, criterion, optimizer)

        # 학습되 모델 저장
        modelfilename = Cmm.GetSaveModelName("CFNeuralnet")
        modelpath = Cmm.GetSaveModelPath() + modelfilename
        saveModel = joblib.dump(model, modelpath)
        str64 = Cmm.FileToBase64(modelpath)

        if os.path.exists(modelpath):
            os.remove(modelpath)

        # 테스트 데이터로 예측 결과 반환
        predarray = model.predict(X_test)
        pred_y_test = predarray['Predarray']

        #softmax 사용하여 확률 값으로 변환
        anomalyScore = np.round(F.softmax(pred_y_test['Predarray'], dim=0).detach().numpy(), 3)

        # 파이토치 텐서를 넘파이 어레이로 전환
        # pred_y_test = pred_y_test.detach().numpy()
        # pred_prob = pred_y_test
        #
        # fpr_array = np.array([], dtype=np.int32)
        # tpr_array = np.array([], dtype=np.int32)
        #
        # for i in range(1, 100):
        #     predd = np.array([], dtype=np.int32)
        #     for j in range(len(pred_prob)):
        #         decision_boundary = i / 100
        #         if (pred_prob[j] > decision_boundary) == True:
        #             predd = np.append(predd, 1)
        #         else:
        #             predd = np.append(predd, 0)
        #
        #     cofMatt = confusion_matrix(y_test, predd, labels=[1, 0])  # 1: 불량, 0: 정상
        #     ##confusion matrix의 각 요소
        #     tpp = cofMatt[1, 1]
        #     tnn = cofMatt[0, 0]
        #     fnn = cofMatt[1, 0]
        #     fpp = cofMatt[0, 1]
        #     tpr = tpp / (tpp + fnn)
        #     fpr = fpp / (tnn + fpp)
        #
        #     fpr_array = np.append(fpr_array, fpr)
        #     tpr_array = np.append(tpr_array, tpr)


        # calculate AUC of model
        fpr, tpr, thresholds = roc_curve(y_test, pred_y_test)
        auc = roc_auc_score(y_test, pred_y_test)

        lists = fpr.tolist()
        fprJson = Cmm.NumlistToArraystring(lists)
        lists = tpr.tolist()
        tprJson = Cmm.NumlistToArraystring(lists)
        lists = pred_y_test.tolist()
        pred_y_testJson = Cmm.NumlistToOneValueArraystring(lists)

        return ({"AnomalyScore":anomalyScore, "Aucscore": str(auc), "Fpr": fprJson, "Tpr": tprJson, "Predarray" :pred_y_testJson}), {"Modelfile": str(modelfilename), "Modelcontents": str(str64)}

# trdataid=20220707-0003-CS&tsdataid=20220707-0003-CT&cols=쿠션위치,보압절환위치,계량완료위치,형계위치,사출최대속도&fcols=불량판정&hiddenlayer=2&node=3&ouputnode=5&epochs=10
# a = Classification.cnetexcute('SET2022000007-CS','SET2022000007-CT', '계량완료위치,사출최대속도','판정코드',2,5,1,10)
# print(a)
#
#
# df1 = pd.read_csv("C:\\Classfication\\ab_te.csv")
# x = df1.iloc[:,[0, 1, 2, 3]]
# y = df1.iloc[:,[4]]
# X_train,X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=33)
# pred_y1_test = Classification.CNet(X_train, y_train, X_test,y_test, 2, 5, 1, 10)
# print(pred_y1_test)
# pred_y1_test

# pred_y1_test['Fpr']
# pred_y1_test['Tpr']

#
#
# CNET = torch.load("CNet.pt")
# CNET.eval()
# X_test = torch.Tensor(X_test.values)
# pred_y_test = CNET.predict(X_test)
# print(pred_y_test)




In [43]:
testY

102    0
75     1
37     1
133    0
125    0
116    0
2      1
63     1
43     1
11     1
81     1
85     1
23     1
27     1
95     1
79     1
52     1
143    0
13     1
21     1
24     1
57     1
41     1
68     1
111    0
18     1
122    0
104    0
74     1
132    0
56     1
25     1
39     1
106    0
69     1
145    0
91     1
78     1
96     1
131    0
7      1
128    0
134    0
127    0
147    0
Name: label, dtype: object

In [ ]:
import numpy as np
import pandas as pd
from XNCmmLib import XNCmmUnit as Cmm
import XNDatabaseLib as DbLib

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score,roc_curve
import joblib
import os

class Classification:
    def randomforestexcute(trdataID, tsdataID, cols, fcols, criterion, max_depth, n_estimators):
        # TrainingID를 dataframe 변수
        txt = DbLib.DatabaseUnit.GetDataSet(trdataID)
        df = pd.read_csv(Cmm.StrToStringIO(txt), sep=',', encoding='utf-8-sig')
        trdf = Cmm.DfToCustomDf(df, Cmm.StrToList(cols, ','))
        # print(trdf)

        # 불량판정 데이터열을 dataframe 변수
        trfdf = Cmm.DfToCustomDf(df, Cmm.StrToList(fcols, ','))
        # print(trfdf)

        # 테스트할 ID 값을 dataframe 변수
        txt = DbLib.DatabaseUnit.GetDataSet(tsdataID)
        df = pd.read_csv(Cmm.StrToStringIO(txt), sep=',', encoding='utf-8-sig')
        tsdf = Cmm.DfToCustomDf(df, Cmm.StrToList(cols, ','))
        # print(tsdf)

        tsfdf = Cmm.DfToCustomDf(df, Cmm.StrToList(fcols, ','))
        # print(tsfdf)

        # cbm 호출
        a = Classification.classification_randomforest(trdf, tsdf ,trfdf, tsfdf, criterion, max_depth, n_estimators)
        jstr = Cmm.ClassToJson(a)

        return jstr

    def classification_randomforest(trainX, testX, trainY, testY,criterion='gini', max_depth=None, n_estimators=100 ):
        """

        Parameters
        ----------
        trainX : array
            Train X.
        testX : array
            Test X.
        trainY : array
            Train Y. 이진(binary) 범주형 데이터
        testY : array
            Test Y. 이진(binary) 범저형 데이터
        n_estimators : int, optional, default = 100
            의사 결정 나무의 갯수
        max_depth : int, optional, default = None
            의사 결정 나무의 최대 깊이. 의사 결정 나무가 최대 깊이에 도달하면 더 이상 분기하지 않음.
            None 입력 시 각 노드에 1개의 샘플이 남을 때까지 분기함
        criterion : {'gini', 'entropy', 'log_loss'}, default = "gini"
            의사 결정 나무의 분기 기준.

        Returns
        -------
        model :
            RandomForest 모델.
        confusionMatrix : array
            분류 성능 평가를 위한 Confusion Matrix
        fpr : array
            False Positive Rate
        tpr : array
            True Positive Rate
        auc_score : float
            fpr, tpr의 AUC(Area Under Cruve) scroe (0~1). 1에 가까울 수록 분류 성능이 좋음을 의미함
        accuracy : float
            accuracy.
        precision : float
            precision
        recall : float
            recall
        f1 : float
            f1 score
        tp : int
            True positive
        fp : int
            False positive
        fn : int
            False negative
        tn : int
            True negative

        """
        # 데이터가 DataFrame이면 array타입으로 변경
        if isinstance(trainX, pd.DataFrame):
            trainX = trainX.values
        if isinstance(testX, pd.DataFrame):
            testX = testX.values
        if isinstance(trainY, pd.DataFrame):
            trainY = trainY.values
        if isinstance(testY, pd.DataFrame):
            testY = testY.values

        # 모델링
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion)
        model.fit(trainX, trainY)

        # Random Forest 모델 Pickle 파일로 저장
        modelfilename = Cmm.GetSaveModelName("CFRandomforest")
        modelpath = Cmm.GetSaveModelPath() + modelfilename
        saveModel = joblib.dump(model, modelpath)
        str64 = Cmm.FileToBase64(modelpath)

        if os.path.exists(modelpath):
            os.remove(modelpath)

        # 예측 값
        pred = model.predict(testX)
        anomalyScore = model.predict_proba(testX)

        predarray = Cmm.ListToOneValueArraystring(pred)

        # 분류 평가 지표
        cofMat = confusion_matrix(testY, pred, labels=[1, 0])  # 1: 불량, 0: 정상
        # confusion matrix의 각 요소
        tp = cofMat[1, 1]
        tn = cofMat[0, 0]
        fn = cofMat[1, 0]
        fp = cofMat[0, 1]

        accuracy = accuracy_score(testY, pred)
        precision = precision_score(testY, pred)
        recall = recall_score(testY, pred)
        f1 = f1_score(testY, pred)

        # calculate AUC of model
        auc = roc_auc_score(testY, pred)
        #fpr, tpr, thresholds = roc_curve(testY, pred)

        fpr_array = np.array([], dtype=np.int32)
        tpr_array = np.array([], dtype=np.int32)

        for i in range(1, 100):
            predd = np.array([], dtype=np.int32)
            for j in range(len(anomalyScore)):
                decision_boundary = i / 100
                if (anomalyScore[j][1] > decision_boundary) == True:
                    predd = np.append(predd, 1)
                else:
                    predd = np.append(predd, 0)

            cofMatt = confusion_matrix(testY, predd, labels=[1, 0])  # 1: 불량, 0: 정상
            ##confusion matrix의 각 요소
            tpp = cofMatt[1, 1]
            tnn = cofMatt[0, 0]
            fnn = cofMatt[1, 0]
            fpp = cofMatt[0, 1]
            tpr = tpp / (tpp + fnn)
            fpr = fpp / (tnn + fpp)

            fpr_array = np.append(fpr_array, fpr)
            tpr_array = np.append(tpr_array, tpr)

        fpr_arrayJson = Cmm.ListToOneValueArraystring(fpr_array)
        tpr_arrayJson = Cmm.ListToOneValueArraystring(tpr_array)

        fpr, tpr, thresholds = roc_curve(testY, pred)

        lists = cofMat.tolist()
        cofMatJson = Cmm.NumlistToArraystring(lists)
        lists = fpr.tolist()
        fprJson = Cmm.NumlistToArraystring(lists)
        lists = tpr.tolist()
        tprJson = Cmm.NumlistToArraystring(lists)

        anomalyScore = np.round(anomalyScore[:, 1], 3)
        lists = anomalyScore.tolist()
        anomalyScoreJson = Cmm.NumlistToArraystring(lists)
        # print(anomalyScoreJson)

        return ({"AnomalyScore":anomalyScoreJson, "Aucscore": str(auc),"Fpr": fprJson, "Tpr": tprJson, "Fprarray": fpr_arrayJson, "Tprarray": tpr_arrayJson, "Predarray" : predarray}), {"Modelfile": str(modelfilename), "Modelcontents": str(str64)}
                # "model": model, "accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "tp": tp, "fp": fp, "fn": fn,
                # "tn": tn})  # type : int32

def classification_model_load_rf(model, testX):
    pred = model.predict(testX)
    anomalyScore = model.predict_proba(testX)
    predarray = Cmm.ListToOneValueArraystring(pred)

    # 분류 평가 지표
    # cofMat = confusion_matrix(testY, pred, labels=[1, 0])  # 1: 불량, 0: 정상
    # # confusion matrix의 각 요소
    # tp = cofMat[1, 1]
    # tn = cofMat[0, 0]
    # fn = cofMat[1, 0]
    # fp = cofMat[0, 1]
    #
    # accuracy = accuracy_score(testY, pred)
    # precision = precision_score(testY, pred)
    # recall = recall_score(testY, pred)
    # f1 = f1_score(testY, pred)
    #
    # # calculate AUC of model
    # auc = roc_auc_score(testY, pred)
    # # fpr, tpr, thresholds = roc_curve(testY, pred)
    #
    # fpr_array = np.array([], dtype=np.int32)
    # tpr_array = np.array([], dtype=np.int32)
    #
    # for i in range(1, 100):
    #     predd = np.array([], dtype=np.int32)
    #     for j in range(len(pred_prob)):
    #         decision_boundary = i / 100
    #         if (pred_prob[j][1] > decision_boundary) == True:
    #             predd = np.append(predd, 1)
    #         else:
    #             predd = np.append(predd, 0)
    #
    #     cofMatt = confusion_matrix(testY, predd, labels=[1, 0])  # 1: 불량, 0: 정상
    #     ##confusion matrix의 각 요소
    #     tpp = cofMatt[1, 1]
    #     tnn = cofMatt[0, 0]
    #     fnn = cofMatt[1, 0]
    #     fpp = cofMatt[0, 1]
    #     tpr = tpp / (tpp + fnn)
    #     fpr = fpp / (tnn + fpp)
    #
    #     fpr_array = np.append(fpr_array, fpr)
    #     tpr_array = np.append(tpr_array, tpr)
    #
    # fpr_arrayJson = Cmm.ListToOneValueArraystring(fpr_array)
    # tpr_arrayJson = Cmm.ListToOneValueArraystring(tpr_array)
    #
    # fpr, tpr, thresholds = roc_curve(testY, pred)

    # lists = cofMat.tolist()
    # cofMatJson = Cmm.NumlistToArraystring(lists)
    # lists = fpr.tolist()
    # fprJson = Cmm.NumlistToArraystring(lists)
    # lists = tpr.tolist()
    # tprJson = Cmm.NumlistToArraystring(lists)

    return {"Predarray": predarray}
    # return ({"Aucscore": str(auc), "Fpr": fprJson, "Tpr": tprJson, "Fprarray": fpr_arrayJson, "Tprarray": tpr_arrayJson,
    #          "Predarray": predarray})#, {"Modelfile": str(modelfilename), "Modelcontents": str(str64)}
    # "model": model, "accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "tp": tp, "fp": fp, "fn": fn,
    # "tn": tn})  # type : int32

#
# a = Classification.randomforestexcute('SET2022000017-CS','SET2022000017-CT', '계량완료위치,사출최대속도,사출최대압력','판정코드','gini', None, 100)
# print(a)

# ##예제
# ab_tr = pd.read_csv('C:\\Classfication\\ab_tr.csv')
# ab_te = pd.read_csv('C:\\Classfication\\ab_te.csv')
#
# # ab_tr = ab_tr.drop(['Unnamed: 0'],axis=1)
# # ab_te = ab_te.drop(['Unnamed: 0'],axis=1)
#
# ab_tr_x = ab_tr.drop(['target'],axis=1)
# ab_te_x = ab_te.drop(['target'],axis=1)
# # print(ab_tr_x)
# print(ab_te_x)
# ab_tr_y = ab_tr['target']
# ab_te_y = ab_te['target']
# # print(ab_tr_y)
# print(ab_te_y)

# ab_dt = Classification.classification_randomforest(ab_tr_x, ab_te_x, ab_tr_y, ab_te_y)
# print(ab_dt)

#rf_model = joblib.load()
#print(classification_model_load_KN(rf_model,ab_te_x,ab_te_y))

# rf_model = joblib.load('C:\\ModelStorage\\220718174100297622_CFRandomforest.pkl')
# print(classification_model_load_KN(rf_model,,))


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,f1_score,roc_curve
import numpy as np
from sklearn import svm
import pandas as pd
from XNCmmLib import XNCmmUnit as Cmm
import XNDatabaseLib as DbLib
import joblib
import os

class Classification:
    def svmexcute(trdataID, tsdataID, cols, fcols):
        # TrainingID의 cols열을 dataframe 변수
        txt = DbLib.DatabaseUnit.GetDataSet(trdataID)
        df = pd.read_csv(Cmm.StrToStringIO(txt), sep=',', encoding='utf-8-sig')
        trdf = Cmm.DfToCustomDf(df, Cmm.StrToList(cols, ','))
        # print(trdf)
        # 예측하는 데이터열을 dataframe 변수
        trfdf = Cmm.DfToCustomDf(df, Cmm.StrToList(fcols, ','))
        # print(trfdf)
        txt = DbLib.DatabaseUnit.GetDataSet(tsdataID)
        df = pd.read_csv(Cmm.StrToStringIO(txt), sep=',', encoding='utf-8-sig')
        tsdf = Cmm.DfToCustomDf(df, Cmm.StrToList(cols, ','))
        # print(tsdf)
        tsfdf = Cmm.DfToCustomDf(df, Cmm.StrToList(fcols, ','))
        # print(tsfdf)
        # cbm 호출
        a = Classification.svm_classification(trdf, trfdf ,tsdf, tsfdf)
        jstr = Cmm.ClassToJson(a)

        return jstr

    def svm_classification(trainX, trainY, testX, testY):
        if isinstance(trainX, pd.DataFrame):
            trainX = trainX.values
        if isinstance(testX, pd.DataFrame):
            testX = testX.values
        if isinstance(trainY, pd.DataFrame):
            trainY = trainY.values
        if isinstance(testY, pd.DataFrame):
            testY = testY.values

        model = svm.SVC(kernel='rbf',probability=True).fit(trainX, trainY)

        # SVM 모델 Pickle 파일로 저장
        modelfilename = Cmm.GetSaveModelName("CFSvm")
        modelpath = Cmm.GetSaveModelPath() + modelfilename
        saveModel = joblib.dump(model, modelpath)
        str64 = Cmm.FileToBase64(modelpath)

        if os.path.exists(modelpath):
            os.remove(modelpath)

        anomalyScore = model.predict_proba(testX)
        pred = model.predict(testX)
        predarray = Cmm.ListToOneValueArraystring(pred)

        fpr_array = np.array([], dtype=np.int32)
        tpr_array = np.array([], dtype=np.int32)

        for i in range(1, 100):
            predd = np.array([], dtype=np.int32)
            for j in range(len(anomalyScore)):
                decision_boundary = i / 100
                if (anomalyScore[j][1] > decision_boundary) == True:
                    predd = np.append(predd, 1)
                else:
                    predd = np.append(predd, 0)

            cofMatt = confusion_matrix(testY, predd, labels=[1, 0])  # 1: 불량, 0: 정상
            ##confusion matrix의 각 요소
            tpp = cofMatt[1, 1]
            tnn = cofMatt[0, 0]
            fnn = cofMatt[1, 0]
            fpp = cofMatt[0, 1]
            tpr = tpp / (tpp + fnn)
            fpr = fpp / (tnn + fpp)

            fpr_array = np.append(fpr_array, fpr)
            tpr_array = np.append(tpr_array, tpr)

        #fpr, tpr, threshold = roc_curve(testY, pred)
        auc = roc_auc_score(testY, pred)

        fpr, tpr, thresholds = roc_curve(testY, pred)

        lists = fpr.tolist()
        fprJson = Cmm.NumlistToArraystring(lists)
        lists = tpr.tolist()
        tprJson = Cmm.NumlistToArraystring(lists)

        anomalyScore = np.round(anomalyScore[:, 1],3)
        lists = anomalyScore.tolist()
        anomalyScoreJson = Cmm.NumlistToArraystring(lists)

        fpr_arrayJson = Cmm.ListToOneValueArraystring(fpr_array)
        tpr_arrayJson = Cmm.ListToOneValueArraystring(tpr_array)

        return ({"AnomalyScore":anomalyScoreJson, "Aucscore": str(auc),"Fpr": fprJson, "Tpr": tprJson, "Fprarray": fpr_arrayJson, "Tprarray": tpr_arrayJson, "Predarray" : predarray}), {"Modelfile": str(modelfilename), "Modelcontents": str(str64)}

def classification_model_load_svm(model, testX):
    anomalyScore = model.predict_proba(testX)
    pred = model.predict(testX)
    predarray = Cmm.ListToOneValueArraystring(pred)

    # fpr_array = np.array([], dtype=np.int32)
    # tpr_array = np.array([], dtype=np.int32)
    #
    # for i in range(1, 100):
    #     predd = np.array([], dtype=np.int32)
    #     for j in range(len(pred_prob)):
    #         decision_boundary = i / 100
    #         if (pred_prob[j][1] > decision_boundary) == True:
    #             predd = np.append(predd, 1)
    #         else:
    #             predd = np.append(predd, 0)
    #
    #     cofMatt = confusion_matrix(testY, predd, labels=[1, 0])  # 1: 불량, 0: 정상
    #     ##confusion matrix의 각 요소
    #     tpp = cofMatt[1, 1]
    #     tnn = cofMatt[0, 0]
    #     fnn = cofMatt[1, 0]
    #     fpp = cofMatt[0, 1]
    #     tpr = tpp / (tpp + fnn)
    #     fpr = fpp / (tnn + fpp)
    #
    #     fpr_array = np.append(fpr_array, fpr)
    #     tpr_array = np.append(tpr_array, tpr)
    #
    # # fpr, tpr, threshold = roc_curve(testY, pred)
    # auc = roc_auc_score(testY, pred)
    #
    # fpr, tpr, thresholds = roc_curve(testY, pred)
    #
    # lists = fpr.tolist()
    # fprJson = Cmm.NumlistToArraystring(lists)
    # lists = tpr.tolist()
    # tprJson = Cmm.NumlistToArraystring(lists)
    #
    # fpr_arrayJson = Cmm.ListToOneValueArraystring(fpr_array)
    # tpr_arrayJson = Cmm.ListToOneValueArraystring(tpr_array)

    return {"Predarray": predarray}
    # return ({"Aucscore": str(auc), "Fpr": fprJson, "Tpr": tprJson, "Fprarray": fpr_arrayJson, "Tprarray": tpr_arrayJson,
    #          "Predarray": predarray})#, {"Modelfile": str(modelfilename), "Modelcontents": str(str64)}

# classification-svm?trdataid=20220707-0003-CS&tsdataid=20220707-0003-CT&cols=계량완료위치,쿠션위치,보압절환위치&fcols=불량판정
# a = Classification.svmexcute('SET2022000017-CS','SET2022000017-CT', '계량완료위치,사출최대속도,사출최대압력','판정코드')
# print(a)

# ## 예제
# ab_tr = pd.read_csv('C:\\Classfication\\ab_tr.csv')
# ab_te = pd.read_csv('C:\\Classfication\\ab_te.csv')
#
#
# trainX = ab_tr.drop(['target'],axis=1)
# trainY = ab_tr['target']
#
# testX = ab_te.drop(['target'],axis=1)
# testY = ab_te['target']
#
# ab_svm = Classification.svm_classification(trainX,trainY,testX,testY)
# print(ab_svm)


#svm_model = joblib.load()
#print(classification_model_load_KN(svm_model,testX,testY))